<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       ModelOps demo - DataRobot with ModelOps BYOM no code
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

![image](images/byom_meth.png) 

<p style = 'font-size:20px;font-family:Arial'><b>Introduction:</b></p>
<p style = 'font-size:16px;font-family:Arial'>
This Notebook will show you how to work with the Bring Your Own Model (BYOM) pattern and BYOM In-Vantage Scoring. This deployment pattern allows you to use whatever data science platform you want to perform model training and import back to Vantage to do a direct deployment, scoring. For understanding more about BYOM you can review official user documentation.<br>
When you use BYOM mechanisms you can deploy models directly from IDE environment, or make a proper operationalization through ModelOps. With ModelOps, you get <b>full governance</b> around your models deployment and you can <b>leverage ModelOps automation</b> for Validation, Scoring and Monitoring with an intuitive user interface that let you audit at any time all the information from your models and provide dashboards to monitor and review alerts from your deployments.<br>This notebook will cover the Operationalization of the PIMA diabetes use case with DataRobot standard BYOM format. <b>DataRobot</b> is a widely used analytics application and there is good possibility that a Teradata customer is already using it for their data analytics projects.By bringing analytics to the data, customers who have ML models trained using DataRobot can deploy these models in Vantage and instantly realize value by predicting/scoring the models at scale.</p>


<p style = 'font-size:18px;font-family:Arial'><b>Steps in this Notebook</b></p>
<ol style = 'font-size:16px;font-family:Arial'>
 <li>Configure the Environment </li>
 <li>Connect to Vantage</li>
 <li>Train a model in DataRobot  
 <li>Import the DataRobot into ModelOps</li>
 <li>Go through Lifecycle - Evaluation (Automated Model Report)</li>
 <li>Go through Lifecycle - Approve </li>
 <li>Go through Lifecycle - Deploy (Publish and Schedule)</li>
 <li>Go through Lifecycle - Monitor (Data Drift and Performance)</li>
 <li>Configure Monitoring alert threshold (Optional) </li>
 <li>On demand Scoring from SQL (Optional)</li>
</ol>

<hr style="height:2px;border:none;">
<p><b style = 'font-size:20px;font-family:Arial'>1. Configure the Environment</b></p>

<p style = 'font-size:16px;font-family:Arial'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

<p style = 'font-size:18px;font-family:Arial'><b>1.1 Libraries installation</b></p>

<p style = 'font-size:16px;font-family:Arial'><b>A restart of the Kernel is needed to confirm changes</b>. We use -q parameter for a non-verbose log of the installation command, you may remove this parameter if you want to know all the steps of the pip installation.</p>

In [ ]:
#%pip install -q teradataml==17.20.0.6 teradatamodelops==7.0.3 nyoka==4.3.0

<p style = 'font-size:16px;font-family:Arial'><b>Hint:</b><i>The easy way to restart the kernel to bring the above installed software into memory is to type zero zero (<b> 0 0 </b>).</i></p>

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>1.2 Libraries import</b></p>

In [ ]:
from teradataml import *
from teradatasqlalchemy.types import *
import os
import getpass
import logging
import sys


<hr style="height:2px;border:none;">
<p><b style = 'font-size:20px;font-family:Arial'>2. Connect to Vantage</b></p>

<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, then use down arrow to go to next cell. Begin running steps with Shift + Enter keys.</p>

In [ ]:
%run -i ../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username = 'demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=18_ModelOps_BYOM_PIMA_DataRobot;' UPDATE FOR SESSION;''')

<p style = 'font-size:18px;font-family:Arial'><b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. You can either run the demo using foreign tables to access the data without any storage on your environment or download the data to local storage, which may yield faster execution. Still, there could be considerations of available storage. Two statements are in the following cell, and one is commented out. You may switch which mode you choose by changing the comment string.</p>

In [ ]:
#%run -i ../UseCases/run_procedure.py "call get_data('DEMO_ModelOps_cloud');"        # Takes 10 seconds
%run -i ../UseCases/run_procedure.py "call get_data('DEMO_ModelOps_local');"        # Takes 30 seconds

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>Creating predictions and model table</b></p>
<p style = 'font-size:16px;font-family:Arial'>We will create a predictions table where we get our model predictions and the model table where we will upload the model created.</p>

In [ ]:
#ddl for Aoa_Byom_Models 
query = '''CREATE SET TABLE Aoa_Byom_Models 
     (
      model_version VARCHAR(255) CHARACTER SET LATIN NOT CASESPECIFIC,
      model_id VARCHAR(255) CHARACTER SET LATIN NOT CASESPECIFIC,
      model_type VARCHAR(255) CHARACTER SET LATIN NOT CASESPECIFIC,
      project_id VARCHAR(255) CHARACTER SET LATIN NOT CASESPECIFIC,
      deployed_at TIMESTAMP(6) DEFAULT CURRENT_TIMESTAMP(6),
      model BLOB(2097088000))
UNIQUE PRIMARY INDEX ( model_version );
'''
 
try:
    execute_sql(query)
except:
    db_drop_table('Aoa_Byom_Models')
    execute_sql(query)

In [ ]:
#ddl for Pima_Patient_Predictions
query = '''CREATE MULTISET TABLE Pima_Patient_Predictions 
     (
      job_id VARCHAR(255) CHARACTER SET LATIN NOT CASESPECIFIC,
      PatientId BIGINT,
      HasDiabetes BIGINT,
      json_report CLOB(1048544000) CHARACTER SET LATIN)
PRIMARY INDEX ( job_id );;
'''
 
try:
    execute_sql(query)
except:
    db_drop_table('Pima_Patient_Predictions')
    execute_sql(query)

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../UseCases/run_procedure.py "call space_report();"        # Takes 10 seconds

<hr style="height:2px;border:none;">
<p><b style = 'font-size:20px;font-family:Arial'>3. Train a model in DataRobot</b></p>

<p style = 'font-size:16px;font-family:Arial'>DataRobot is a widely used analytics application and there is good possibility that a Teradata customer is already using it for their data analytics projects.<br>By bringing analytics to the data, customers who have ML models trained using DataRobot can deploy these models in Vantage and instantly realize value by predicting/scoring the models at scale. This reduces cost as it eliminates  costly data export steps are eliminated and model prediction in Vantage is proven to be multiple times faster than other platforms and data is secure.<br>DataRobot charges its users for the deployment of trained ML models externally to Vantage. Every ML model deployed externally counts against the user/customer DataRobot pre-paid licenses. Once the DataRobot license limit is met, the user/customer must purchase additional licenses before it can deploy more models
Once a model and its scoring code is downloaded, it counts against the available license permanently and cannot be reversed or deleted or updated. Below message appears when the user downloads a DataRobot Model with scoring code.
</p>
<img src="images/datarobot.png" alt="DataRobot download model" />


<p style = 'font-size:16px;font-family:Arial'>For the demo purpose we have created a model in DataRobot using the PIMA diabetes dataset and have downloaded the same. The model is placed in the artifacts folder for the next steps please you can use model.datarobot if you dont have your own datarobot subscription.</p> 

<hr style="height:2px;border:none;">
<p><b style = 'font-size:20px;font-family:Arial'>4. Import into ModelOps to Operationalize</b></p>

<p style = 'font-size:16px;font-family:Arial'>Go to the ModelOps UI and import this as a new model inside the <b>Demo</b> project.<br>Navigate to the projects screen and select the <b>Demo</b> project. Click on it to navigate to the models screen of the project.</p>

<img src="images/18_1.png" alt="Projects screen" />

<p style = 'font-size:16px;font-family:Arial'>Once there, click on the <b>DEFINE BYOM MODEL</b> button on the top right of the screen.

<img src="images/18_2.png" alt="Models screen" />

<p style = 'font-size:16px;font-family:Arial'>Ensure that <b>Enable Model Monitoring</b> is checked and classification as model type is selected. Then set the table the one you have been using (probably your username) and use as Prediction Expression

```sql
CAST(CAST(json_report AS JSON).JSONExtractValue('$.predicted_HasDiabetes') AS INT)
```

<p style = 'font-size:16px;font-family:Arial'>This prediction expression can be generated with the generator: select the variable: predicted_HasDiabetes from the screen and confirm<br>

<img src="images/18_3.png" alt="BYOM model import sidesheeet" />

<p style = 'font-size:16px;font-family:Arial'>Then click on <b>SAVE</b>, and the import job will start.

<img src="images/18_4.png" alt="BYOM model import job log" />

<p style = 'font-size:16px;font-family:Arial'>Go and try this Step by yourself. Launch ModelOps from this button below:</p>

<a href="/modelops" style="display: inline-flex; align-items: center; justify-content: center; background-color: #017373; color: #FFFFFF; font-family: Arial, sans-serif; font-size: 16px; font-weight: bold; text-decoration: none; padding: 12px 24px; border: none; border-radius: 8px; box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1); ursor: pointer; transition: all 0.3s ease;">
  LAUNCH MODELOPS
  <img src="https://img.icons8.com/ios-filled/50/ffffff/external-link.png" alt="External Link Icon" style="margin-left: 8px; width: 20px; height: 20px;">
</a>

<hr style="height:2px;border:none;">
<p><b style = 'font-size:20px;font-family:Arial'>5. Go through Lifecycle - Evaluation (Automated Model Report)</b></p>

<p style = 'font-size:16px;font-family:Arial'>Open the imported ModelID Lifecycle Screen. Review imported artifacts and evaluate the model with default logic (No code needed)</p>
<img src="images/18_5.png" alt="BYOM model lifecycle"/>
<p style = 'font-size:16px;font-family:Arial'>Select Evaluation dataset and run the evaluation process.</p>
                              
<img src="images/18_6.png" alt="Evaluation" width="500" height="500"/>

<p style = 'font-size:16px;font-family:Arial'>Now you can review the Model evaluation report generated with default metrics and confusion matrix plot - this can be customized with a custom evaluation logic (later in this notebook)<br>Click on View Report from ModelID Lifecycle screen.</p>

<img src="images/18_7.png" alt="Model Evaluation Report"/>

<p style = 'font-size:16px;font-family:Arial'>Go and try this Step by yourself. Launch ModelOps from this button below:</p>

<a href="/modelops" style="display: inline-flex; align-items: center; justify-content: center; background-color: #017373; color: #FFFFFF; font-family: Arial, sans-serif; font-size: 16px; font-weight: bold; text-decoration: none; padding: 12px 24px; border: none; border-radius: 8px; box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1); ursor: pointer; transition: all 0.3s ease;">
  LAUNCH MODELOPS
  <img src="https://img.icons8.com/ios-filled/50/ffffff/external-link.png" alt="External Link Icon" style="margin-left: 8px; width: 20px; height: 20px;">
</a>

<hr style="height:2px;border:none;">
<p><b style = 'font-size:20px;font-family:Arial'>6. Go through Lifecycle - Approve </b></p>

<p style = 'font-size:16px;font-family:Arial'>Open the imported ModelID Lifecycle Screen. Click on Approve button to move forward on the next stage.<br>Include description of approval and accept</p>

<img src="images/18_8.png" alt="Approval" width="500" height="500"/>

<p style = 'font-size:16px;font-family:Arial'>Go and try this Step by yourself. Launch ModelOps from this button below:</p>

<a href="/modelops" style="display: inline-flex; align-items: center; justify-content: center; background-color: #017373; color: #FFFFFF; font-family: Arial, sans-serif; font-size: 16px; font-weight: bold; text-decoration: none; padding: 12px 24px; border: none; border-radius: 8px; box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1); ursor: pointer; transition: all 0.3s ease;">
  LAUNCH MODELOPS
  <img src="https://img.icons8.com/ios-filled/50/ffffff/external-link.png" alt="External Link Icon" style="margin-left: 8px; width: 20px; height: 20px;">
</a>

<hr style="height:2px;border:none;">
<p><b style = 'font-size:20px;font-family:Arial'>7. Go through Lifecycle - Deploy (Publish and Schedule)</b></p>

<p style = 'font-size:16px;font-family:Arial'>Open the imported ModelID Lifecycle Screen. Click on Deploy button to publish Model in Vantage.</p>
<img src="images/18_9.png" alt="Model Lifecycle screen"/>

<p style = 'font-size:16px;font-family:Arial'>Now select In-Vantage engine and click next</p>

<img src="images/18_10.png" alt="Deployment Engine" width="500" height="500"/>

<p style = 'font-size:16px;font-family:Arial'>Now select which Connection (here you could use a Service Connection to run in Production optionally), Database and table you want to publish your BYOM model. Use your user "demo_user" and the table "aoa_byom_models"</p>

<img src="images/18_11.png" alt="Deployment Publish" width="500" height="500"/>

<p style = 'font-size:16px;font-family:Arial'>Now select if you want to schedule your model scoring, dataset template to gather where you want to store your predictions and run the deploy process</p>

<img src="images/18_12.png" alt="Deployment Schedule" width="500" height="500"/>

<p style = 'font-size:16px;font-family:Arial'>Go and try this Step by yourself. Launch ModelOps from this button below:</p>

<a href="/modelops" style="display: inline-flex; align-items: center; justify-content: center; background-color: #017373; color: #FFFFFF; font-family: Arial, sans-serif; font-size: 16px; font-weight: bold; text-decoration: none; padding: 12px 24px; border: none; border-radius: 8px; box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1); ursor: pointer; transition: all 0.3s ease;">
  LAUNCH MODELOPS
  <img src="https://img.icons8.com/ios-filled/50/ffffff/external-link.png" alt="External Link Icon" style="margin-left: 8px; width: 20px; height: 20px;">
</a>

<hr style="height:2px;border:none;">
<p><b style = 'font-size:20px;font-family:Arial'>8. Go through Lifecycle - Monitor (Data Drift and Performance)</b></p>

<p style = 'font-size:16px;font-family:Arial'>Open the Deployment and review details of the deployed model.</p>
<img src="images/18_13.png" alt="Deployment details"/>
<p style = 'font-size:16px;font-family:Arial'>You can run the Prediction job from here , click the button to run the prediction job</p>
<img src="images/18_14.png" alt="Deployment Scoring Job" />
<p style = 'font-size:16px;font-family:Arial'>You can also review the predictions, you can take this query and run into your SQL IDE or in Notebook later.</p>
<img src="images/18_15.png" alt="Deployment Predictions" width="500" height="500"/> <img src="images/18_16.png" alt="Deployment Predictions query" width="500" height="500"/>
<p style = 'font-size:16px;font-family:Arial'>Review your Feature Drift, Prediction Drift. Here ModelOps shows the comparison of the distribution of data between training and evaluation/scoring, every scoring this gets updated and different KPIs are calculated. The one we used by default for monitoring is the Population Stability Index (PSI).</p>

<img src="images/18_17.png" alt="Feature Drift" />
<img src="images/18_18.png" alt="Prediction Drift" />


<p style = 'font-size:16px;font-family:Arial'>For Performance Monitoring, we track over time the metrics of the model. For generating new metrics we need to create a new evaluation dataset and run an evaluation job.</p>

<img src="images/18_19.png" alt="Performance Drift" />

<p style = 'font-size:16px;font-family:Arial'>This has been generated using this query for evaluation dataset target:</p>

```sql
SELECT * FROM Demo_ModelOps.pima_patient_diagnoses F WHERE F.patientid MOD 8 <> 0 
```    
<p style = 'font-size:16px;font-family:Arial'>Now the performance has changed:</p>

<img src="images/18_20.png" alt="Performance Drift change" />

<p style = 'font-size:16px;font-family:Arial'>Go and try this Step by yourself. Launch ModelOps from this button below:</p>

<a href="/modelops" style="display: inline-flex; align-items: center; justify-content: center; background-color: #017373; color: #FFFFFF; font-family: Arial, sans-serif; font-size: 16px; font-weight: bold; text-decoration: none; padding: 12px 24px; border: none; border-radius: 8px; box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1); ursor: pointer; transition: all 0.3s ease;">
  LAUNCH MODELOPS
  <img src="https://img.icons8.com/ios-filled/50/ffffff/external-link.png" alt="External Link Icon" style="margin-left: 8px; width: 20px; height: 20px;">
</a>

<hr style="height:2px;border:none;">
<p><b style = 'font-size:20px;font-family:Arial'>9. Configure Monitoring alert threshold (Optional)</b></p>

<p style = 'font-size:16px;font-family:Arial'>
We can update the Alerts configuration for the models.<br>First enable Alert from Model catalog</p>

<img src="images/18_21.png" alt="Alert Configuration" /> 

<p style = 'font-size:16px;font-family:Arial'>Now, you can change your default alerting mechanism.<br>Go to your Model, and find the Tab "Alert Configuration"</p>

<img src="images/18_22.png" alt="Alert Configuration" /> 

<p style = 'font-size:16px;font-family:Arial'>Let's change the PSI value to 0.01 instead 0.2 <br>Use Edit option:</p>

<img src="images/18_23.png" alt="Alert Configuration PSI" width="500" height="500"/> 
<img src="images/18_24.png" alt="Alert Configuration PSI" width="500" height="500"/> 

<p style = 'font-size:16px;font-family:Arial'>After few minutes, this has generated new alerts on the model deployed
Go to Menu -> Alert and review the alerts generated for your model</p>

<img src="images/18_25.png" alt="Alerts" />

<p style = 'font-size:16px;font-family:Arial'>Now go to ModelID and click on View Model Drift</p>

<img src="images/18_26.png" alt="Model Lifecycle View Model Drift" width="500" height="500"/>

<p style = 'font-size:16px;font-family:Arial'>Now review the Feature drift, you can check this screen also from your deployments</p>
<img src="images/18_27.png" alt="Feature Drift" />

<p style = 'font-size:16px;font-family:Arial'>Go and try this Step by yourself. Launch ModelOps from this button below:</p>

<a href="/modelops" style="display: inline-flex; align-items: center; justify-content: center; background-color: #017373; color: #FFFFFF; font-family: Arial, sans-serif; font-size: 16px; font-weight: bold; text-decoration: none; padding: 12px 24px; border: none; border-radius: 8px; box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1); ursor: pointer; transition: all 0.3s ease;">
  LAUNCH MODELOPS
  <img src="https://img.icons8.com/ios-filled/50/ffffff/external-link.png" alt="External Link Icon" style="margin-left: 8px; width: 20px; height: 20px;">
</a>

<hr style="height:2px;border:none;">
<p><b style = 'font-size:20px;font-family:Arial'>10. On demand Scoring from SQL (Optional)</b></p>

<p><b style = 'font-size:18px;font-family:Arial'>10.1 View Published Models</b></p>
<p style = 'font-size:16px;font-family:Arial'>
Once deployed via ModelOps, we can view the models published to vantage by querying the table they are published to. Note this information is available via the AOA APIs also.</p>

In [ ]:
DataFrame.from_query("SELECT TOP 10 * FROM Demo_User.aoa_byom_models WHERE model_type='DATAROBOT'").head(10)

<hr style="height:1px;border:none;">
<p><b style = 'font-size:18px;font-family:Arial'>10.2 On-Demand Scoring</b></p>
<p style = 'font-size:16px;font-family:Arial'>Configuring VAL and BYOM locations:</p>

In [ ]:
# configure byom/val installation
configure.val_install_location = "VAL"
configure.byom_install_location = "MLDB"


<p style = 'font-size:16px;font-family:Arial'>Replace model_version with the one we have operationalized:</p>

In [ ]:
model_version="b7dfab89-5194-442f-a5ff-35b56841124a"

model = DataFrame.from_query(f"""
SELECT * FROM aoa_byom_models 
    WHERE model_version='{model_version}'
""")


preds = DataRobotPredict(
        modeldata=model,
        newdata=DataFrame.from_query("SELECT * FROM Demo_ModelOps.pima_patient_features WHERE patientid MOD 5 = 0"),
        accumulate=['PatientId'])

preds.result.head(10)

In [ ]:
#SQL version
query = f"""
SELECT * FROM MLDB.DataRobotPredict (
    ON (SELECT * FROM Demo_ModelOps.pima_patient_features WHERE patientid MOD 5 = 0) AS InputTable
    ON (SELECT * FROM aoa_byom_models 
            WHERE model_version='{model_version}') AS ModelTable DIMENSION
    USING
      Accumulate ('patientid')
) AS td;
"""

DataFrame.from_query(query).head(10)

<hr style="height:2px;border:none;">
<p><b style = 'font-size:20px;font-family:Arial'>11. Cleanup</b></p>

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial'>If you are done with ModelOps usecase, please uncomment and run the below cleanup section.</p>
</div>

<p style = 'font-size:18px;font-family:Arial'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>Cleanup work tables to prevent errors next time.</p>

In [ ]:
# db_drop_table(table_name = 'aoa_byom_models', schema_name = 'demo_user')
# db_drop_table(table_name = 'pima_patient_predictions', schema_name = 'demo_user')

<p style = 'font-size:18px;font-family:Arial'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
# %run -i ../UseCases/run_procedure.py "call remove_data('DEMO_ModelOps');"        # Takes 10 seconds

In [ ]:
remove_context()

<a href="./00_ModelOps_Introduction.ipynb" style="font-size:16px; font-family:Arial; color:white; background-color:#007bff; padding:10px 20px; border-radius:5px; text-decoration:none; display:inline-blok;">
  &lt;&lt; Back to Introduction
</a>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>